In [1]:
import numpy as np
import pandas as pd

In [2]:
interactions_path = "./data/RAW_interactions.csv"
recipes_path = "./data/RAW_recipes.csv"

### Loading Data

In [3]:
interactions_raw = pd.read_csv(interactions_path)
recipes_raw = pd.read_csv(recipes_path)

In [4]:
interactions_raw.head(1)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...


In [5]:
recipes_raw.head(1)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7


In [6]:
interactions_raw.groupby('recipe_id').head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."
...,...,...,...,...,...
1132362,116593,72730,2003-12-09,0,Another approach is to start making sauce with...
1132363,583662,386618,2009-09-29,5,These were so delicious! My husband and I tru...
1132364,157126,78003,2008-06-23,5,WOW! Sometimes I don't take the time to rate ...
1132365,53932,78003,2009-01-11,4,Very good! I used regular port as well. The ...


In [7]:
interactions_raw.groupby(['recipe_id', 'rating'])['user_id'].agg('count').reset_index()

,recipe_id,rating,user_id
0,38,4,3
1,38,5,1
2,39,3,1
3,40,1,1
4,40,4,2
...,...,...,...
367682,537459,5,1
367683,537485,5,1
367684,537543,0,1
367685,537671,0,1


In [17]:
K = 6
def count_ratings(x):
    ret = [0] * K
    ratings = list(x.rating)
    counts = list(x.user_id)
    for rating, count in zip(ratings, counts):
        ret[K - rating - 1] = count
    return ret

In [18]:
recipes_with_rating_counts = interactions_raw.groupby(['recipe_id', 'rating'])['user_id'].agg('count').reset_index().groupby('recipe_id')[['rating', 'user_id']].apply(count_ratings).reset_index(name='ratings')

In [19]:
import math
def starsort(ns):
    """
    http://www.evanmiller.org/ranking-items-with-star-ratings.html
    """
    N = sum(ns)
    K = len(ns)
    s = list(range(K,0,-1))
    s2 = [sk**2 for sk in s]
    z = 1.65
    def f(s, ns):
        N = sum(ns)
        K = len(ns)
        return sum(sk*(nk+1) for sk, nk in zip(s,ns)) / (N+K)
    fsns = f(s, ns)
    return fsns - z*math.sqrt((f(s2, ns)- fsns**2)/(N+K+1))

In [20]:
recipes_with_rating_counts['popularity'] = recipes_with_rating_counts['ratings'].apply(starsort)

In [21]:
recipes_with_rating_counts.head()

,recipe_id,ratings,popularity
0,38,"[1, 3, 0, 0, 0, 0]",3.404010
1,39,"[0, 0, 1, 0, 0, 0]",2.643421
2,40,"[6, 2, 0, 0, 1, 0]",3.896731
3,41,"[1, 1, 0, 0, 0, 0]",3.047372
4,43,"[0, 0, 0, 0, 1, 0]",2.313839


In [22]:
recipes_with_rating_counts.sort_values('popularity', ascending=False).head()

,recipe_id,ratings,popularity
224586,486261,"[215, 2, 0, 0, 0, 0]",5.868667
23999,42976,"[112, 5, 0, 0, 0, 1]",5.679877
53087,92072,"[121, 11, 1, 0, 0, 1]",5.654607
13495,25723,"[161, 23, 1, 2, 0, 1]",5.647498
42195,73488,"[117, 16, 1, 1, 0, 0]",5.643824


In [ ]:
# Get recommended recipes for a given user id - call popularity model with recipes_with_rating_counts and a given user id


#### Get the top 10 most popular recipes

In [23]:
n = 10
top_n_recipes = list(recipes_with_rating_counts.sort_values('popularity', ascending=False).head(n).recipe_id)

In [24]:
top_n_recipes

[486261, 42976, 92072, 25723, 73488, 69173, 100526, 106251, 8739, 31128]

In [25]:
recipes_raw.set_index('id').loc[top_n_recipes].reset_index(inplace=False)

,id,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,486261,mexican stack up rsc,75,226863,2012-09-01,"['time-to-make', 'course', 'main-ingredient', ...","[793.0, 83.0, 10.0, 34.0, 87.0, 131.0, 11.0]",28,"['preheat broiler to high broil', 'set the rac...","ready, set, cook! reynolds wrap contest entry....","['poblano pepper', 'reynolds wrap foil', 'oliv...",22
1,42976,brown sugar bundt cake,90,35047,2002-10-14,"['weeknight', 'time-to-make', 'course', 'cuisi...","[637.4, 57.0, 211.0, 21.0, 11.0, 25.0, 24.0]",9,['grease a 10-inch bundt pan well and sprinkle...,my mom passed this recipe along to me. it is e...,"['yellow cake mix', 'instant vanilla flavor pu...",7
2,92072,monterey chicken rolls,75,76951,2004-05-28,"['time-to-make', 'course', 'main-ingredient', ...","[455.6, 42.0, 4.0, 30.0, 78.0, 55.0, 3.0]",6,"['pound chicken breasts flat', 'slice monterey...",butter...garlic...cheese? chicken rolled in al...,"['boneless skinless chicken breasts', 'montere...",7
3,25723,tiny cinnamon rolls,42,37779,2002-04-20,"['60-minutes-or-less', 'time-to-make', 'course...","[44.2, 1.0, 11.0, 2.0, 2.0, 0.0, 2.0]",10,"['unroll dough , and separate into 4 rectangle...",southern living. my kids adore these tiny litt...,"['refrigerated crescent dinner rolls', 'sugar'...",6
4,73488,individual cream cheese danish,40,101823,2003-10-16,"['60-minutes-or-less', 'time-to-make', 'course...","[87.6, 4.0, 24.0, 3.0, 3.0, 7.0, 4.0]",8,"['preheat oven to 350 degrees f', 'slice cresc...",small danishes that make a great addition to a...,"['crescent roll dough', 'cream cheese', 'white...",9
5,69173,kittencal s italian melt in your mouth meatballs,50,89831,2003-08-20,"['60-minutes-or-less', 'time-to-make', 'course...","[1312.6, 129.0, 8.0, 108.0, 214.0, 174.0, 8.0]",5,['mix all ingredients together in a large bowl...,cooking the meatballs in simmering pasta sauce...,"['ground beef', 'egg', 'parmesan cheese', 'bre...",10
6,100526,sweet creamy coleslaw,15,1535,2004-09-24,"['15-minutes-or-less', 'time-to-make', 'course...","[216.1, 20.0, 50.0, 13.0, 4.0, 9.0, 8.0]",6,"['shred cabbage and carrots into a bowl', 'mix...",i never liked coleslaw until i tried my husban...,"['green cabbage', 'carrots', 'mayonnaise', 'vi...",5
7,106251,roasted cauliflower 16 roasted cloves of garlic,40,58104,2004-12-16,"['60-minutes-or-less', 'time-to-make', 'course...","[126.9, 14.0, 11.0, 17.0, 6.0, 6.0, 3.0]",4,"['mix oil , rosemary , salt , pepper and garli...","i find it totally amazing, that children and m...","['cauliflower', 'garlic cloves', 'fresh rosema...",6
8,8739,wholly guacamole,20,7802,2001-04-09,"['30-minutes-or-less', 'time-to-make', 'course...","[500.7, 68.0, 14.0, 0.0, 13.0, 32.0, 10.0]",13,"['cut avocados in half , remove pit , and spoo...",this is a true tex-mex recipe concocted by a d...,"['avocados', 'garlic cloves', 'lime', 'tomatoe...",8
9,31128,yummy crunchy apple pie,70,29450,2002-06-13,"['weeknight', 'time-to-make', 'course', 'main-...","[360.6, 22.0, 161.0, 6.0, 5.0, 38.0, 19.0]",15,['use foil on bottom of oven rather than a coo...,great pie!,"['pastry for single-crust pie', 'sugar', 'flou...",12
